In [1]:
import pandas as pd
import openpyxl

In [2]:
# Read xlsx file and conver to dataframe
df_emitidos = pd.read_excel('Emisiones.xlsx', sheet_name='Report')
df_anulados = pd.read_excel('Anulaciones.xlsx', sheet_name='Report')
df_recuperos = pd.read_excel('Recuperos.xlsx', sheet_name='Report')

In [3]:
tasa = 4.5

In [4]:
def filter_dataframe_vida(df, year, riesgo, vida=False):
    # Initialize an empty list to store the selected DataFrames
    selected_dfs = []

    # Flags to track whether "TOTAL:" and "Contrato:" are found
    total_found = False
    contrato_found = False

    # Iterate through the DataFrame
    for i in range(len(df)):
        # Check conditions for each row
        # Qué secciones calcular: en este caso se excluyen VIDA y CAUCION
        if (
            df.iloc[i, 2] == "TOTAL:" and
            df.iloc[i, 3] == year and
            (df.iloc[i, 7]).split()[0] == "VIDA" and
            not (df.iloc[i, 7]).split()[0] == "CAUCION" and
            df.iloc[i, 4] == riesgo
        ):
            total_found = True
            contrato_found = False
        elif df.iloc[i, 2] == "Contrato:":
            contrato_found = True
            total_found = False
    
        # Select rows between "TOTAL:" and "Contrato:"
        if total_found and not contrato_found:
            selected_dfs.append(df.iloc[i])

    # Check if there are selected DataFrames
    if selected_dfs:
        # Concatenate the list of selected DataFrames into a single DataFrame
        result_df = pd.concat(selected_dfs, axis=1).T

        # Filter out rows that start with specific words in the first column
        result_df = result_df[~result_df.iloc[:, 0].astype(str).str.startswith(('Póliza', 'Operador:'))]

        # Filter out rows that in the third column start with specific words
        result_df = result_df[~result_df.iloc[:, 2].astype(str).str.startswith(('REGIONAL', 'Listado', 'Emisiones', 'Anulaciones'))]

        # Remove columns with all NaN values
        result_df = result_df.dropna(axis=1, how='all')

        # Change the column names for letters. Example: "Unamed: 0" to "A"
        result_df.columns = [chr(65 + i) for i in range(len(result_df.columns))]

        return result_df
    else:
        print("No matching rows found.")
        return None
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_filtered_df = filter_dataframe_vida(df, 2023, 'EXCEDENTE')
if your_filtered_df is not None:
    print(your_filtered_df)
"""


def filter_dataframe_patrimoniales(df, year, riesgo, vida=False):
    # Initialize an empty list to store the selected DataFrames
    selected_dfs = []

    # Flags to track whether "TOTAL:" and "Contrato:" are found
    total_found = False
    contrato_found = False

    # Iterate through the DataFrame
    for i in range(len(df)):
        # Check conditions for each row
        # Qué secciones calcular: en este caso se excluyen VIDA y CAUCION
        if (
            df.iloc[i, 2] == "TOTAL:" and
            df.iloc[i, 3] == year and
            not (df.iloc[i, 7]).split()[0] == "VIDA" and
            not (df.iloc[i, 7]).split()[0] == "CAUCION" and
            df.iloc[i, 4] == riesgo
        ):
            total_found = True
            contrato_found = False
        elif df.iloc[i, 2] == "Contrato:":
            contrato_found = True
            total_found = False
    
        # Select rows between "TOTAL:" and "Contrato:"
        if total_found and not contrato_found:
            selected_dfs.append(df.iloc[i])

    # Check if there are selected DataFrames
    if selected_dfs:
        # Concatenate the list of selected DataFrames into a single DataFrame
        result_df = pd.concat(selected_dfs, axis=1).T

        # Filter out rows that start with specific words in the first column
        result_df = result_df[~result_df.iloc[:, 0].astype(str).str.startswith(('Póliza', 'Operador:'))]

        # Filter out rows that in the third column start with specific words
        result_df = result_df[~result_df.iloc[:, 2].astype(str).str.startswith(('REGIONAL', 'Listado', 'Emisiones', 'Anulaciones'))]

        # Remove columns with all NaN values
        result_df = result_df.dropna(axis=1, how='all')

        # Change the column names for letters. Example: "Unamed: 0" to "A"
        result_df.columns = [chr(65 + i) for i in range(len(result_df.columns))]

        return result_df
    else:
        print("No matching rows found.")
        return None
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_filtered_df = filter_dataframe_patrimoniales(df, 2023, 'EXCEDENTE')
if your_filtered_df is not None:
    print(your_filtered_df)
"""


def filter_recuperos_patrimoniales(df, year, riesgo, vida=False):
    # Initialize an empty list to store the selected DataFrames
    selected_dfs = []

    # Flags to track whether "TOTAL:" and "Contrato:" are found
    total_found = False
    contrato_found = False

    # Iterate through the DataFrame
    for i in range(len(df)):
        # Check conditions for each row
        # Qué secciones calcular: en este caso se excluyen VIDA y CAUCION
        if (
            df.iloc[i, 0] == "Total Contrato:" and
            df.iloc[i, 2] == year and
            not (df.iloc[i, 9]).split()[0] == "VIDA" and
            not (df.iloc[i, 9]).split()[0] == "CAUCION" and
            df.iloc[i, 3] == riesgo
        ):
            total_found = True
            contrato_found = False
        elif df.iloc[i, 0] == "Contrato:" or df.iloc[i, 1] == "Resumen":
            contrato_found = True
            total_found = False
    
        # Select rows between "TOTAL:" and "Contrato:"
        if total_found and not contrato_found:
            selected_dfs.append(df.iloc[i])

    # Check if there are selected DataFrames
    if selected_dfs:
        # Concatenate the list of selected DataFrames into a single DataFrame
        result_df = pd.concat(selected_dfs, axis=1).T

        # Filter out rows that start with specific words in the first column
        result_df = result_df[~result_df.iloc[:, 0].astype(str).str.startswith(('Póliza', 'Operador:'))]

        # Filter out rows that in the third column start with specific words
        result_df = result_df[~result_df.iloc[:, 2].astype(str).str.startswith(('REGIONAL', 'Listado', 'Emisiones', 'Anulaciones'))]

        # Remove columns with all NaN values
        result_df = result_df.dropna(axis=1, how='all')

        # Change the column names for letters. Example: "Unamed: 0" to "A"
        result_df.columns = [chr(65 + i) for i in range(len(result_df.columns))]

        return result_df
    else:
        print("No matching rows found.")
        return None
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_filtered_df = filter_recuperos_patrimoniales(df, 2023, 'EXCEDENTE')
if your_filtered_df is not None:
    print(your_filtered_df)
"""


def filter_recuperos_vida(df, year, riesgo, vida=False):
    # Initialize an empty list to store the selected DataFrames
    selected_dfs = []

    # Flags to track whether "TOTAL:" and "Contrato:" are found
    total_found = False
    contrato_found = False

    # Iterate through the DataFrame
    for i in range(len(df)):
        # Check conditions for each row
        # Qué secciones calcular: en este caso se excluyen VIDA y CAUCION
        if (
            df.iloc[i, 0] == "Total Contrato:" and
            df.iloc[i, 2] == year and
            (df.iloc[i, 9]).split()[0] == "VIDA" and
            not (df.iloc[i, 9]).split()[0] == "CAUCION" and
            df.iloc[i, 3] == riesgo
        ):
            total_found = True
            contrato_found = False
        elif df.iloc[i, 0] == "Contrato:" or df.iloc[i, 1] == "Resumen":
            contrato_found = True
            total_found = False
    
        # Select rows between "TOTAL:" and "Contrato:"
        if total_found and not contrato_found:
            selected_dfs.append(df.iloc[i])

    # Check if there are selected DataFrames
    if selected_dfs:
        # Concatenate the list of selected DataFrames into a single DataFrame
        result_df = pd.concat(selected_dfs, axis=1).T

        # Filter out rows that start with specific words in the first column
        result_df = result_df[~result_df.iloc[:, 0].astype(str).str.startswith(('Póliza', 'Operador:'))]

        # Filter out rows that in the third column start with specific words
        result_df = result_df[~result_df.iloc[:, 2].astype(str).str.startswith(('REGIONAL', 'Listado', 'Emisiones', 'Anulaciones'))]

        # Remove columns with all NaN values
        result_df = result_df.dropna(axis=1, how='all')

        # Change the column names for letters. Example: "Unamed: 0" to "A"
        result_df.columns = [chr(65 + i) for i in range(len(result_df.columns))]

        return result_df
    else:
        print("No matching rows found.")
        return None
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_filtered_df = filter_recuperos_patrimoniales(df, 2023, 'EXCEDENTE')
if your_filtered_df is not None:
    print(your_filtered_df)
"""


def remove_totals(df):
    try:
        # Remove all the rows that start with "TOTAL:" in the second column
        result_df = df[~df.iloc[:, 1].astype(str).str.startswith('TOTAL:')]
        
        # Remove all empty rows
        result_df = result_df.dropna(how='all')

        return result_df
    except Exception as e:
        print("No matches found.")
        return None

"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
your_processed_df = process_dataframe(your_dataframe)
print(your_processed_df)
"""


def remove_totals_recuperos(df):
    try:
        # Remove all the rows that start with "Total Contrato:" in the first column
        result_df = df[~df.iloc[:, 0].astype(str).str.startswith('Total Contrato:')]
        
        # Remove all empty rows
        result_df = result_df.dropna(how='all')

        return result_df
    except Exception as e:
        print("No matches found.")
        return None
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
your_processed_df = remove_totals_recuperos(your_dataframe)
print(your_processed_df)
"""


def calculate_sums(df):
    try:
        # Suma los valores numéricos de las columnas H, la columna L y la columna M,
        # devuelve los valores y los almacena en variables "prima", "importe_comision" y "a_favor" respectivamente
        prima = pd.to_numeric(df['H'], errors='coerce').fillna(0).sum()
        importe_comision = pd.to_numeric(df['L'], errors='coerce').fillna(0).sum()
        a_favor = pd.to_numeric(df['M'], errors='coerce').fillna(0).sum()

        # Return the calculated values as a dictionary
        return {
            'prima': prima,
            'importe_comision': importe_comision,
            'a_favor': a_favor
        }
    except Exception as e:
        print("No matches found. Values set to 0")
        return {
            'prima': 0,
            'importe_comision': 0,
            'a_favor': 0
        }
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
your_processed_df = process_dataframe(your_dataframe)
prima_value, importe_comision_value, a_favor_value = calculate_sums(your_processed_df)
print(prima_value, importe_comision_value, a_favor_value)
"""


def calculate_sums_recuperos(df):
    try:
        # Suma los valores numéricos de las columnas J, la columna M y la columna O,
        # devuelve los valores y los almacena en variables "indemnizacion", "gastos" y "importe_total" respectivamente
        indemnizacion = pd.to_numeric(df['J'], errors='coerce').fillna(0).sum()
        gastos = pd.to_numeric(df['M'], errors='coerce').fillna(0).sum()
        importe_total = pd.to_numeric(df['O'], errors='coerce').fillna(0).sum()

        # Return the calculated values as a dictionary
        return {
            'indemnizacion': indemnizacion,
            'gastos': gastos,
            'importe_total': importe_total
        }
    except Exception as e:
        print("No matches found. Values set to 0")
        return {
            'indemnizacion': 0,
            'gastos': 0,
            'importe_total': 0
        }

In [29]:
def create_reaseguros_dict(df):
    # Elimina las filas que tienen la palabra "Reasegurador" en la primera columna
    filtered_df = df[df.iloc[:, 0] != 'Reasegurador']

    # Obtén los valores únicos de la primera columna
    unique_values_A = filtered_df.iloc[:, 0].unique()

    # Crea un diccionario con valores de la primera columna como keys y sumas de las columnas 8, 12, 13 como values
    result_dict = {}
    for value in unique_values_A:
        if pd.notna(value):  # Check if the value is not null or NaN
            prima = filtered_df.loc[filtered_df.iloc[:, 0] == value, filtered_df.columns[7]].sum()
            importe_comision = filtered_df.loc[filtered_df.iloc[:, 0] == value, filtered_df.columns[11]].sum()
            a_favor = filtered_df.loc[filtered_df.iloc[:, 0] == value, filtered_df.columns[12]].sum()
            result_dict[value] = {'prima': prima, 'importe_comision': importe_comision, 'a_favor': a_favor}

    return result_dict

def create_reaseguros_dict_recuperos(df):
    # Elimina las filas que tienen la palabra "Reasegurador" en la primera columna
    filtered_df = df[df.iloc[:, 4] != 'Reasegurador']

    # Obtén los valores únicos de la primera columna
    unique_values_A = filtered_df.iloc[:, 4].unique()

    # Crea un diccionario con valores de la primera columna como keys y valores de la quinta columna como values
    result_dict = {}
    for value in unique_values_A:
        # Verifica si la fila no está vacía antes de acceder a la quinta columna
        if not filtered_df.loc[filtered_df.iloc[:, 4] == value, filtered_df.columns[5]].empty:
            result_dict[value] = filtered_df.loc[filtered_df.iloc[:, 4] == value, filtered_df.columns[6]].values[0]
    
    return result_dict

In [6]:
def process_and_sum_vida(df, year, riesgo):
    # Step 1: Filter the DataFrame
    filtered_df = filter_dataframe_vida(df, year, riesgo)

    # Step 2: Generate reaseguros dictionary
    reaseguros_dict = create_reaseguros_dict(filtered_df)
    
    # Step 3: Remove totals from the filtered DataFrame
    processed_df = remove_totals(filtered_df)
    
    # Step 4: Calculate the sums
    sums_result = calculate_sums(processed_df)
    
    return sums_result
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_sums_result = process_and_sum_vida(df, 2023, 'EXCEDENTE')
print("Prima:", your_sums_result[0])
print("Importe Comision:", your_sums_result[1])
print("A Favor:", your_sums_result[2])
"""


def process_and_sum_patrimoniales(df, year, riesgo):
    # Step 1: Filter the DataFrame
    filtered_df = filter_dataframe_patrimoniales(df, year, riesgo)

    # Step 2: Generate reaseguros dictionary
    reaseguros_dict = create_reaseguros_dict(filtered_df)
    
    # Step 2: Remove totals from the filtered DataFrame
    processed_df = remove_totals(filtered_df)
    
    # Step 3: Calculate the sums
    sums_result = calculate_sums(processed_df)
    
    return sums_result, reaseguros_dict
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_sums_result = process_and_sum_patrimoniales(df, 2023, 'EXCEDENTE')
print("Prima:", your_sums_result[0])
print("Importe Comision:", your_sums_result[1])
print("A Favor:", your_sums_result[2])
"""


def process_sum_recuperos_patrimoniales(df, year, riesgo):
    # Step 1: Filter the DataFrame
    filtered_df = filter_recuperos_patrimoniales(df, year, riesgo)

    # Step 2: Generate reaseguros dictionary
    reaseguros_dict = create_reaseguros_dict_recuperos(filtered_df)
    
    # Step 3: Remove totals from the filtered DataFrame
    processed_df = remove_totals_recuperos(filtered_df)
    
    # Step 4: Calculate the sums
    sums_result = calculate_sums_recuperos(processed_df)
    
    return sums_result, reaseguros_dict
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_sums_result = process_sum_recuperos_patrimoniales(df, 2023, 'EXCEDENTE')
print("Indemnizacion:", your_sums_result[0])
print("Gastos:", your_sums_result[1])
print("Importe Total:", your_sums_result[2])
"""


def process_sum_recuperos_vida(df, year, riesgo):
    # Step 1: Filter the DataFrame
    filtered_df = filter_recuperos_vida(df, year, riesgo)

    # Step 2: Generate reaseguros dictionary
    reaseguros_dict = create_reaseguros_dict_recuperos(filtered_df)
    
    # Step 3: Remove totals from the filtered DataFrame
    processed_df = remove_totals_recuperos(filtered_df)
    
    # Step 4: Calculate the sums
    sums_result = calculate_sums_recuperos(processed_df)
    
    return sums_result, reaseguros_dict
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_sums_result = process_sum_recuperos_vida(df, 2023, 'EXCEDENTE')
print("Indemnizacion:", your_sums_result[0])
print("Gastos:", your_sums_result[1])
print("Importe Total:", your_sums_result[2])
"""


'\n# Example usage\n# Replace \'your_dataframe\' with the actual DataFrame variable name\n# Replace 2022 and \'EXCEDENTE\' with the desired values for year and riesgo\nyour_sums_result = process_sum_recuperos_vida(df, 2023, \'EXCEDENTE\')\nprint("Indemnizacion:", your_sums_result[0])\nprint("Gastos:", your_sums_result[1])\nprint("Importe Total:", your_sums_result[2])\n'

In [7]:
def calculate_table_values(prima_qs, prima_exc, prima_anulada_qs, prima_anulada_exc,
                               comisiones_qs, comisiones_exc, siniestros_qs, siniestros_exc, tasa):
    # Calculate the values
    primas_cedidas = prima_qs + prima_exc
    primas_anuladas = prima_anulada_qs + prima_anulada_exc
    comisiones = comisiones_qs + comisiones_exc
    siniestros_pagados = siniestros_qs + siniestros_exc
    impuestos = (primas_cedidas - (primas_anuladas + comisiones)) * tasa
    balance_a_favor_debe = primas_anuladas + comisiones + siniestros_pagados + impuestos
    balance_a_favor_haber = primas_cedidas

    # Return the calculated values
    return primas_cedidas, primas_anuladas, comisiones, siniestros_pagados, impuestos, balance_a_favor_debe, balance_a_favor_haber
"""
# Example usage:
# Replace the parameter values with actual values
result_values = calculate_table_values(prima_qs, prima_exc, prima_anulada_qs, prima_anulada_exc,
                                            comisiones_qs, comisiones_exc, siniestros_qs, siniestros_exc, tasa)
"""

'\n# Example usage:\n# Replace the parameter values with actual values\nresult_values = calculate_table_values(prima_qs, prima_exc, prima_anulada_qs, prima_anulada_exc,\n                                            comisiones_qs, comisiones_exc, siniestros_qs, siniestros_exc, tasa)\n'

In [22]:
test_df=filter_dataframe_patrimoniales(df_emitidos, 2023, 'EXCEDENTE')

In [23]:
test_df.to_excel('test_emitidos_df.xlsx', index=False)

In [30]:
print(create_reaseguros_dict(test_df))

{'MS AMLIN AG': {'prima': 1087343123, 'importe_comision': 337076291, 'a_favor': 750266832}, 'SCOR REINSURANCE COMPANY': {'prima': 434937249, 'importe_comision': 108734290, 'a_favor': 326202959}, 'KOREAN REINSURANCE COMPANY': {'prima': 326202936, 'importe_comision': 110908972, 'a_favor': 215293964}, 'MAPFRE RE COMPAÑIA DE REASEGUROS S.A.': {'prima': 217468624, 'importe_comision': 69589944, 'a_favor': 147878680}, 'REASEGURADORA PATRIA S.A.': {'prima': 108734311, 'importe_comision': 30445601, 'a_favor': 78288710}}


In [11]:
emitidos_cuota_parte_2021 = process_and_sum_patrimoniales(df_emitidos, 2021, 'CUOTA PARTE')
anulados_cuota_parte_2021 = process_and_sum_patrimoniales(df_anulados, 2021, 'CUOTA PARTE')
recuperos_cuota_parte_2021 = process_sum_recuperos_patrimoniales(df_recuperos, 2021, 'CUOTA PARTE')

No matching rows found.


AttributeError: 'NoneType' object has no attribute 'iloc'

In [ ]:
print(emitidos_cuota_parte_2021)
print(anulados_cuota_parte_2021)
print(recuperos_cuota_parte_2021)

{'prima': 0, 'importe_comision': 0, 'a_favor': 0}
{'prima': 4911354.0, 'importe_comision': 1701770.0, 'a_favor': 3209584.0}
{'indemnizacion': 289879239.0, 'gastos': 575051.0, 'importe_total': 290454290.0}


In [ ]:
emitidos_cuota_parte_2023 = process_and_sum_patrimoniales(df_emitidos, 2023, 'CUOTA PARTE')
anulados_cuota_parte_2023 = process_and_sum_patrimoniales(df_anulados, 2023, 'CUOTA PARTE')
recuperos_cuota_parte_2023 = process_sum_recuperos_patrimoniales(df_recuperos, 2023, 'CUOTA PARTE')


In [ ]:
emitidos_excedente_2023 = process_and_sum_patrimoniales(df_emitidos, 2023, 'EXCEDENTE')
anulados_excedente_2023 = process_and_sum_patrimoniales(df_anulados, 2023, 'EXCEDENTE')
recuperos_excedente_2023 = process_sum_recuperos_patrimoniales(df_recuperos, 2023, 'EXCEDENTE')

In [ ]:
# Emitidos CUOTA PARTE 2023
prima_emitidos_qs_2023 = emitidos_cuota_parte_2023['prima']
importe_comision_emitidos_qs_2023 = emitidos_cuota_parte_2023['importe_comision']
# Anulados CUOTA PARTE 2023
prima_anulados_qs_2023 = anulados_cuota_parte_2023['prima']
importe_comision_anulados_qs_2023 = anulados_cuota_parte_2023['importe_comision']
# Recuperos CUOTA PARTE 2023
indemnizacion_recuperos_qs_2023 = recuperos_cuota_parte_2023['indemnizacion']
gastos_recuperos_qs_2023 = recuperos_cuota_parte_2023['gastos']
importe_total_recuperos_qs_2023 = recuperos_cuota_parte_2023['importe_total']

In [ ]:
# Emitidos EXCEDENTE 2023
prima_emitidos_excedente_2023 = emitidos_excedente_2023['prima']
importe_comision_emitidos_excedente_2023 = emitidos_excedente_2023['importe_comision']
# Anulados EXCEDENTE 2023
prima_anulados_excedente_2023 = anulados_excedente_2023['prima']
importe_comision_anulados_excedente_2023 = anulados_excedente_2023['importe_comision']
# Recuperos EXCEDENTE 2023
indemnizacion_recuperos_excedente_2023 = recuperos_excedente_2023['indemnizacion']
gastos_recuperos_excedente_2023 = recuperos_excedente_2023['gastos']
importe_total_recuperos_excedente_2023 = recuperos_excedente_2023['importe_total']

In [ ]:
print("prima_emitidos_excedente_2023:", prima_emitidos_excedente_2023)
print("importe_comision_emitidos_excedente_2023:", importe_comision_emitidos_excedente_2023)
print("prima_anulados_excedente_2023:", prima_anulados_excedente_2023)
print("importe_comision_anulados_excedente_2023:", importe_comision_anulados_excedente_2023)
print("indemnizacion_recuperos_excedente_2023:", indemnizacion_recuperos_excedente_2023)
print("gastos_recuperos_excedente_2023:", gastos_recuperos_excedente_2023)
print("importe_total_recuperos_excedente_2023:", importe_total_recuperos_excedente_2023)

prima_emitidos_excedente_2023: 2174686243.0
importe_comision_emitidos_excedente_2023: 656755098.0
prima_anulados_excedente_2023: 222174885.0
importe_comision_anulados_excedente_2023: 67096913.0
indemnizacion_recuperos_excedente_2023: 638095112.0
gastos_recuperos_excedente_2023: 16133964.0
importe_total_recuperos_excedente_2023: 654229076.0


In [ ]:
prima_qs = prima_emitidos_qs_2023
prima_exc = prima_emitidos_excedente_2023
prima_anulada_qs = prima_anulados_qs_2023
prima_anulada_exc = prima_anulados_excedente_2023
comisiones_qs = importe_comision_emitidos_qs_2023-importe_comision_anulados_qs_2023
comisiones_exc = importe_comision_emitidos_excedente_2023-importe_comision_anulados_excedente_2023
siniestros_qs = indemnizacion_recuperos_qs_2023
siniestros_exc = indemnizacion_recuperos_excedente_2023

In [ ]:
print("prima_qs:", prima_qs)
print("prima_exc:", prima_exc)
print("prima_anulada_qs:", prima_anulada_qs)
print("prima_anulada_exc:", prima_anulada_exc)
print("comisiones_qs:", comisiones_qs)
print("comisiones_exc:", comisiones_exc)
print("siniestros_qs:", siniestros_qs)
print("siniestros_exc:", siniestros_exc)

prima_qs: 2799208423.0
prima_exc: 2174686243.0
prima_anulada_qs: 193979440.0
prima_anulada_exc: 222174885.0
comisiones_qs: 818042261.0
comisiones_exc: 589658185.0
siniestros_qs: 625415637.0
siniestros_exc: 638095112.0


In [ ]:
def calculate_resumen_values(emitidos_qs, anulados_qs, recuperos_qs, emitidos_exc, anulados_exc, recuperos_exc):
    prima_qs = emitidos_qs['prima']
    prima_exc = emitidos_exc['prima']
    prima_anulada_qs = anulados_qs['prima']
    prima_anulada_exc = anulados_exc['prima']
    comisiones_qs = emitidos_qs['importe_comision'] - anulados_qs['importe_comision']
    comisiones_exc = emitidos_exc['importe_comision'] - anulados_exc['importe_comision']
    siniestros_qs = recuperos_qs['indemnizacion']
    siniestros_exc = recuperos_exc['indemnizacion']

    return {
        'prima_qs': prima_qs,
        'prima_exc': prima_exc,
        'prima_anulada_qs': prima_anulada_qs,
        'prima_anulada_exc': prima_anulada_exc,
        'comisiones_qs': comisiones_qs,
        'comisiones_exc': comisiones_exc,
        'siniestros_qs': siniestros_qs,
        'siniestros_exc': siniestros_exc
    }

In [ ]:
# Calclulate resumen values
resumen_values = calculate_resumen_values(emitidos_cuota_parte_2023, anulados_cuota_parte_2023, recuperos_cuota_parte_2023,
                                          emitidos_excedente_2023, anulados_excedente_2023, recuperos_excedente_2023)
print(resumen_values)

{'prima_qs': 2799208423.0, 'prima_exc': 2174686243.0, 'prima_anulada_qs': 193979440.0, 'prima_anulada_exc': 222174885.0, 'comisiones_qs': 818042261.0, 'comisiones_exc': 589658185.0, 'siniestros_qs': 625415637.0, 'siniestros_exc': 638095112.0}


In [ ]:
def calculate_table_values(prima_qs, prima_exc, prima_anulada_qs, prima_anulada_exc,
                               comisiones_qs, comisiones_exc, siniestros_qs, siniestros_exc, tasa):
    # Calculate the values
    primas_cedidas = prima_qs + prima_exc
    primas_anuladas = prima_anulada_qs + prima_anulada_exc
    comisiones = comisiones_qs + comisiones_exc
    siniestros_pagados = siniestros_qs + siniestros_exc
    impuestos = (primas_cedidas - (primas_anuladas + comisiones)) * tasa
    balance_a_favor_debe = primas_anuladas + comisiones + siniestros_pagados + impuestos
    balance_a_favor_haber = primas_cedidas

    # Return the calculated values
    return primas_cedidas, primas_anuladas, comisiones, siniestros_pagados, impuestos, balance_a_favor_debe, balance_a_favor_haber

# Example usage:
# Replace the parameter values with actual values
result_values = calculate_table_values(prima_qs, prima_exc, prima_anulada_qs, prima_anulada_exc,
                                            comisiones_qs, comisiones_exc, siniestros_qs, siniestros_exc, tasa)

# Access the calculated values
primas_cedidas, primas_anuladas, comisiones, siniestros_pagados, impuestos, balance_a_favor_debe, balance_a_favor_haber = result_values

In [ ]:
print(result_values)

(4973894666.0, 416154325.0, 1407700446.0, 1263510749.0, 14175179527.5, 17262545047.5, 4973894666.0)


In [ ]:
"""
prima_qs = "{:,.0f}".format(prima_emitidos_qs_2023).replace(',', '.')
prima_exc = "{:,.0f}".format(prima_emitidos_excedente_2023).replace(',', '.')
prima_anulada_qs = "{:,.0f}".format(prima_anulados_qs_2023).replace(',', '.')
prima_anulada_exc = "{:,.0f}".format(prima_anulados_excedente_2023).replace(',', '.')
comisiones_qs = "{:,.0f}".format(importe_comision_emitidos_qs_2023 - importe_comision_anulados_qs_2023).replace(',', '.')
comisiones_exc = "{:,.0f}".format(importe_comision_emitidos_excedente_2023 - importe_comision_anulados_excedente_2023).replace(',', '.')
siniestros_qs = "{:,.0f}".format(indemnizacion_recuperos_qs_2023).replace(',', '.')
siniestros_exc = "{:,.0f}".format(indemnizacion_recuperos_excedente_2023).replace(',', '.')
"""

'\nprima_qs = "{:,.0f}".format(prima_emitidos_qs_2023).replace(\',\', \'.\')\nprima_exc = "{:,.0f}".format(prima_emitidos_excedente_2023).replace(\',\', \'.\')\nprima_anulada_qs = "{:,.0f}".format(prima_anulados_qs_2023).replace(\',\', \'.\')\nprima_anulada_exc = "{:,.0f}".format(prima_anulados_excedente_2023).replace(\',\', \'.\')\ncomisiones_qs = "{:,.0f}".format(importe_comision_emitidos_qs_2023 - importe_comision_anulados_qs_2023).replace(\',\', \'.\')\ncomisiones_exc = "{:,.0f}".format(importe_comision_emitidos_excedente_2023 - importe_comision_anulados_excedente_2023).replace(\',\', \'.\')\nsiniestros_qs = "{:,.0f}".format(indemnizacion_recuperos_qs_2023).replace(\',\', \'.\')\nsiniestros_exc = "{:,.0f}".format(indemnizacion_recuperos_excedente_2023).replace(\',\', \'.\')\n'

In [ ]:
# Imprimir resultados
print("Vigencia/Contrato: 2023")
print("Prima cedida en el periodo (QS):", prima_qs)
print("Prima cedida en el periodo (EXCEDENTE):", prima_exc)
print("Prima anulada en el periodo (QS):", prima_anulada_qs)
print("Prima anulada en el periodo (EXCEDENTE):", prima_anulada_exc)
print("Comisiones(QS):", (comisiones_qs))
print("Comisiones(EXCEDENTE):", (comisiones_exc)) 
print("Siniestros pagados en el periodos (QS):", siniestros_qs)
print("Siniestros pagados en el periodos (EXCEDENTE):", siniestros_exc)


Vigencia/Contrato: 2023
Prima cedida en el periodo (QS): 2799208423.0
Prima cedida en el periodo (EXCEDENTE): 2174686243.0
Prima anulada en el periodo (QS): 193979440.0
Prima anulada en el periodo (EXCEDENTE): 222174885.0
Comisiones(QS): 818042261.0
Comisiones(EXCEDENTE): 589658185.0
Siniestros pagados en el periodos (QS): 625415637.0
Siniestros pagados en el periodos (EXCEDENTE): 638095112.0


In [ ]:
primas_cedidas = prima_qs + prima_exc
primas_anuladas = prima_anulada_qs + prima_anulada_exc
comisiones = comisiones_qs + comisiones_exc
siniestros_pagados = siniestros_qs + siniestros_exc
impuestos = (primas_cedidas - (primas_anuladas + comisiones)) * tasa
balance_a_favor_debe = primas_anuladas + comisiones + siniestros_pagados + impuestos
balance_a_favor_haber = primas_cedidas

In [ ]:
print("Primas cedidas:", primas_cedidas)
print("Primas anuladas:", primas_anuladas)
print("Comisiones:", comisiones)
print("Siniestros pagados:", siniestros_pagados)
print("Impuestos:", impuestos)
print("Balance a favor (Debe):", balance_a_favor_debe)
print("Balance a favor (Haber):", balance_a_favor_haber)


Primas cedidas: 4973894666.0
Primas anuladas: 416154325.0
Comisiones: 1407700446.0
Siniestros pagados: 1263510749.0
Impuestos: 14175179527.5
Balance a favor (Debe): 17262545047.5
Balance a favor (Haber): 4973894666.0


In [ ]:
def print_results_resumen(year, prima_qs, prima_exc, prima_anulada_qs, prima_anulada_exc,
                  comisiones_qs, comisiones_exc, siniestros_qs, siniestros_exc):
    # Print header
    print(f"Vigencia/Contrato: {year}")
    
    # Define the data to be printed
    data = [
        ("Prima cedida en el periodo (QS):", prima_qs),
        ("Prima cedida en el periodo (EXCEDENTE):", prima_exc),
        ("Prima anulada en el periodo (QS):", prima_anulada_qs),
        ("Prima anulada en el periodo (EXCEDENTE):", prima_anulada_exc),
        ("Comisiones(QS):", comisiones_qs),
        ("Comisiones(EXCEDENTE):", comisiones_exc),
        ("Siniestros pagados en el periodos (QS):", siniestros_qs),
        ("Siniestros pagados en el periodos (EXCEDENTE):", siniestros_exc)
    ]

    # Print the table
    for label, value in data:
        print(f"{label:<45} {value}")

# Example usage
# Replace the variables with actual values
print_results_resumen(2023, prima_qs, prima_exc, prima_anulada_qs, prima_anulada_exc,
              comisiones_qs, comisiones_exc, siniestros_qs, siniestros_exc)

Vigencia/Contrato: 2023
Prima cedida en el periodo (QS):              2799208423.0
Prima cedida en el periodo (EXCEDENTE):       2174686243.0
Prima anulada en el periodo (QS):             193979440.0
Prima anulada en el periodo (EXCEDENTE):      222174885.0
Comisiones(QS):                               818042261.0
Comisiones(EXCEDENTE):                        589658185.0
Siniestros pagados en el periodos (QS):       625415637.0
Siniestros pagados en el periodos (EXCEDENTE): 638095112.0


In [ ]:
def print_debe_haber_table(primas_cedidas, primas_anuladas, comisiones, siniestros_pagados, impuestos, balance_a_favor_debe, balance_a_favor_haber):
    # Print header
    print(f"{'CONCEPTO':<35}{'DEBE':<20}{'HABER':<20}")

    # Define the data to be printed
    data = [
        ("Primas cedidas", "", primas_cedidas),
        ("Primas anuladas", primas_anuladas, ""),
        ("Comisiones", comisiones, ""),
        ("Siniestros pagados", siniestros_pagados, ""),
        ("Impuestos", impuestos, ""),
        ("Balance a favor", balance_a_favor_debe, balance_a_favor_haber)
    ]

    # Print the table
    for concept, debe_value, haber_value in data:
        print(f"{concept:<35}{debe_value:<20}{haber_value:<20}")

# Example usage
# Replace the variables with actual values
print_debe_haber_table(
    primas_cedidas,
    primas_anuladas,
    comisiones,
    siniestros_pagados,
    impuestos,
    balance_a_favor_debe,
    balance_a_favor_haber
)

CONCEPTO                           DEBE                HABER               
Primas cedidas                                         4973894666.0        
Primas anuladas                    416154325.0                             
Comisiones                         1407700446.0                            
Siniestros pagados                 1263510749.0                            
Impuestos                          14175179527.5                           
Balance a favor                    17262545047.5       4973894666.0        
